In [6]:
from dataclasses import dataclass, field
import requests
import datetime as dt


#       ** CLASS STRUCTURES **       #
######################################

# Did not use a Data Class for the library due to the complexity of the class
class StarWarsLibrary:
    """ Star Wars Library Class """
    
    # Variables with _ _ underscores should only be accessible from within the class
    def __init__(self):
        self.__token = 'tt3896198'
        self.__api_key = '3ef42ac3'
        self.__star_path = 'https://swapi.dev/api/films'
        self.__omdb_path = f'http://www.omdbapi.com/?i={self.__token}&apikey={self.__api_key}&t=Star+Wars&y='
        self.__star_data = None
        self.__omdb_data = None
        self.__movie_dict = {}
        self.last_updated = dt.datetime.now() # This will get moved to a function that runs after a successful update
    

    
    # Allows a user to check out a movie object
    def get_movies(self):
        return self.__movie_dict
    
    # API Access Methods
    ######################################
    
    def access_star_wars_api(self):
        self.__star_data = requests.get(self.__star_path).json()['results']
        
    def access_omdb_api(self, year):
        self.__omdb_data = requests.get(self.__omdb_path + year).json()
        
    # Data Parsing
    ######################################
    
    def parse_omdb_data(self):
        omdb_data = self.__omdb_data
        movie_title = omdb_data['Title']
        box_office = omdb_data['BoxOffice'].replace(",","").replace("$","")
        rotten_rating = None
        for rating in omdb_data['Ratings']:
            if rating['Source'] == 'Rotten Tomatoes':
                rotten_rating = rating['Value']
        return [rotten_rating, box_office]
          
    # Data Update Methods    
    ######################################
    
    def update_movie_dict(self,new_movies):
        for movie in new_movies:
            self.__movie_dict[movie.title] = movie
        return self.__movie_dict
        
    def update_library(self):
        print('Checking for new movies.')
        self.access_star_wars_api()
        self.update_film_database()
    
    def update_film_database(self):
        """ 
        Primary Data Gathering Function 
        - Called by class.method()
        - Checks for available updates
        """
        
        films = []
        for i,film in enumerate(self.__star_data):
            print(film)
            # If the film is already collected, skip the film
            if film['title'] in self.__movie_dict:
                continue
            
            print(f'Parsing {film["title"]}')
            film_info = {
                'title': None, 
                'episode_id': None, 
                'opening_crawl': None, 
                'director': None, 
                'producer': None, 
                'release_date': None, 
                'characters': None, 
                'plot': None, 
                'rotten_tomatoes': None, 
                'box_office_gross': None    
                }
            
            for field in film_info.keys():
                if field in film:
                    film_info[field] = film[field]
                    
            year = film_info['release_date'][:4]
            self.access_omdb_api(year)
            omdb_data = self.parse_omdb_data()
            film_info['rotten_tomatoes'] = omdb_data[0]
            film_info['box_office_gross'] = omdb_data[1]
            film_obj = StarWarsFilms(*film_info.values())
            films.append(film_obj)
            print("######################")
        self.update_movie_dict(films)    
        

@dataclass
class StarWarsFilms:     
    
    # This __post_init__ method is checking the input data to see if it is valid.
    # (Might need to be a __init__ instead, but then I'll have to change the class structure)
    def __post_init__(self):
        if self.episode_id is None:
            raise Exception("An Episode ID is required!!")
        
        # This adjusts the character input to be complete character objects instead of links
        if self.characters is not None:
            updated_characters = []
            for character in self.characters:
                char = self.get_characters(character)
                print(char['name'])
                attrs = self.get_character_attributes(char)
                updated_characters.append(FilmCharacters(*attrs))
            self.characters = updated_characters
    
    title: str = None
    episode_id: int = None
    opening_crawl: str = None
    director: str = None
    producer: str = None
    release_date: str = None
    characters: list = None
    plot: str = None
    rotten_tomatoes: str = None
    box_office_gross: int = None #int = field(metadata={"units":"U.S. Dollars"})
    # Can't seem to use metadata with a default value
    
    def __lt__(self,other):
        if self.episode_id < other.episode_id:
            return True
        return False
    
    def __gt__(self,other):
        if self.episode_id > other.episode_id:
            return True
        return False
    
    def __eq__(self,other):
        if self.episode_id == othe.episode_idr:
            return True
        return False
    
    def __ge__(self,other):
        if self.episode_id >= other.episode_id:
            return True
        return False
    
    # This function is calling the api for the character information
    def get_characters(self, character):
        chars = requests.get(character).json()
        return chars
    
    def get_character_attributes(self,character):
        goal_list = [
            'name',
            'height',
            'mass',
            'hair_color',
            'eye_color',
            'birth_year',
            'gender'
        ]
        
        attr_list = []
        for attr in goal_list:
            if attr in character:
                attr_list.append(character[attr])
            else:
                attr_list.append(None)
        return attr_list
    
@dataclass
class FilmCharacters:
    name: str
    height: float
    mass: float = field(metadata={"units":"kilograms"})
    hair_color: str
    eye_color: str
    birth_year: str
    gender: str


        

In [7]:
build = StarWarsLibrary()

In [8]:
build.update_library()

Checking for new movies.
{'title': 'A New Hope', 'episode_id': 4, 'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....", 'director': 'George Lucas', 'producer': 'Gary Kurtz, Rick McCallum', 'release_date': '1977-05-25', 'characters': ['https://swapi.dev/api/people/1/', 'https://swapi.dev/api/people/2/', 'https://swapi.dev/api/people/3/', 'https://swapi.dev/api/people/4/', 'https://swapi.dev/api/people/5/', 'https://swapi.dev/api/people/6/', 'https://swapi.dev/api/peop

In [9]:
movies = build.get_movies()

In [4]:
movies = build.get_movies()

In [19]:
for movie in movies:
    print(movies[movie].title)
    print(movies[movie].rotten_tomatoes)
    print(movies[movie].box_office_gross)
    print("\n")

A New Hope
92%
460998507


The Empire Strikes Back
94%
292753960


Return of the Jedi
82%
309306177


The Phantom Menace
52%
474544677


Attack of the Clones
65%
310676740


Revenge of the Sith
80%
380270577




In [17]:
for movie in movies:
    print(movies[movie])

StarWarsFilms(title='A New Hope', episode_id=4, opening_crawl="It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....", director='George Lucas', producer='Gary Kurtz, Rick McCallum', release_date='1977-05-25', characters=[FilmCharacters(name='Luke Skywalker', height='172', mass='77', hair_color='blond', eye_color='blue', birth_year='19BBY', gender='male'), FilmCharacters(name='C-3PO', height='167', mass='75', hair_color='n/a', eye_color='yellow', birth_year='112BBY', gender='n/a'), FilmC